Author: Nicholas Steele
Snow Survey Hydrologist
USDA - Natural Resources Conservation Service
1201 NE Lloyd Blvd, Suite #900
Portland, OR  97232
Email: nick.steele@usda.gov
Cell: 503-819-5880


# Imports

In [1]:
# from sklearn.linear_model import LassoCV, RidgeCV, HuberRegressor
# from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor
# from sklearn.model_selection import train_test_split
# from sklearn import svm
# from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.stats as stats
# import statsmodels.api as sm
# from functools import reduce

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
import numpy as np
import pandas as pd
from datetime import date
import RegressionToolmultiselect as RegressionTool

import requests
import xmltodict
import datetime

# import RegressionFun

In [2]:
apple = RegressionTool.RegressionFun([{'302:OR:SNTL':'WTEQ'},  {'302:OR:SNTL':'SNWD'}, {'302:OR:SNTL':'TAVG'}, {'653:OR:SNTL':'WTEQ'}], '01/01/2010','02/01/2020')

KeyboardInterrupt: 

In [ ]:
apple.train_model('SVM',0.3)

In [23]:
apple.make_predictions('03/01/2020', '04/01/2020')

# Dash Web App

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# cd /content/gdrive/My Drive/Projects/Work/SNOTELRegressionTool

/content/gdrive/My Drive/Projects/Work/SNOTELRegressionTool


In [11]:
Stations = pd.read_excel('SNOTELStationNames&Triplets.xlsx')
Stations

,Station Name,Station Triplet
0,Adin Mtn,301:CA:SNTL
1,Cedar Pass,391:CA:SNTL
2,Crowder Flat,977:CA:SNTL
3,Dismal Swamp,446:CA:SNTL
4,State Line,1258:CA:SNTL
...,...,...
154,Trough,832:WA:SNTL
155,Upper Wheeler,841:WA:SNTL
156,Waterhole,974:WA:SNTL
157,Wells Creek,909:WA:SNTL


In [5]:
stations = Stations.loc[:,'Station Name'].tolist()
triplets = Stations.loc[:, 'Station Triplet'].tolist()

# stations_complete = dict(zip(stations, triplets))

# labels = {'label': stations}
# values = {'value':triplets}

In [6]:
options = []
for i in range(len(Stations.index)):
  options.append({'label': stations[i], 'value': triplets[i]})

In [7]:
options

[{'label': 'Adin Mtn', 'value': '301:CA:SNTL'},
 {'label': 'Cedar Pass', 'value': '391:CA:SNTL'},
 {'label': 'Crowder Flat', 'value': '977:CA:SNTL'},
 {'label': 'Dismal Swamp', 'value': '446:CA:SNTL'},
 {'label': 'State Line', 'value': '1258:CA:SNTL'},
 {'label': 'Aneroid Lake #2', 'value': '302:OR:SNTL'},
 {'label': 'Annie Springs', 'value': '1000:OR:SNTL'},
 {'label': 'Arbuckle Mtn', 'value': '304:OR:SNTL'},
 {'label': 'Bear Grass', 'value': '1166:OR:SNTL'},
 {'label': 'Beaver Reservoir', 'value': '331:OR:SNTL'},
 {'label': 'Big Red Mountain', 'value': '341:OR:SNTL'},
 {'label': 'Bigelow Camp', 'value': '343:OR:SNTL'},
 {'label': 'Billie Creek Divide', 'value': '344:OR:SNTL'},
 {'label': 'Blazed Alder', 'value': '351:OR:SNTL'},
 {'label': 'Blue Mountain Spring', 'value': '357:OR:SNTL'},
 {'label': 'Bourne', 'value': '361:OR:SNTL'},
 {'label': 'Bowman Springs', 'value': '362:OR:SNTL'},
 {'label': 'Cascade Summit', 'value': '388:OR:SNTL'},
 {'label': 'Chemult Alternate', 'value': '395:

In [ ]:
# import itertools

# def zip_with_scalar(l, o):
#     return zip(l, itertools.repeat(o))

In [ ]:
# key = 'label'
# labels = list(map(lambda i: (key, i), Sites))
# labels

In [3]:
!dir

 Volume in drive C is Windows
 Volume Serial Number is B067-5610

 Directory of C:\Users\nicks\Projects\SNOTELRegressionTool

09/29/2021  11:29 AM    <DIR>          .
09/29/2021  11:29 AM    <DIR>          ..
09/29/2021  10:15 AM    <DIR>          .ipynb_checkpoints
09/29/2021  10:14 AM            15,508 RegressionFun.ipynb
09/29/2021  10:14 AM            11,876 RegressionFun.py
09/28/2021  03:19 PM         2,618,800 RegressionTool_v2_0.ipynb
09/29/2021  11:29 AM            32,235 RegressionTool_v2_0_APP.ipynb
09/28/2021  03:50 PM            14,198 SNOTELStationNames_Triplets.xlsx
09/29/2021  10:15 AM    <DIR>          __pycache__
               5 File(s)      2,692,617 bytes
               4 Dir(s)   6,803,058,688 bytes free


In [12]:
Stations = pd.read_excel('SNOTELStationNames&Triplets.xlsx')

In [46]:

#----------------------------------------------------------------------
# variables to be used in different places within app

heading={'color':'black', 'font-size':50,'text-Align':'center','font-weight': 'bold'}
subheading={'color':'black', 'font-weight': 'bold', 'text-Align':'left'}
station_dropdown_style = {'width': '45%', 'display': 'inline-block', 'float':'left'}
parameter_dropdown_style = {'width': '45%', 'display': 'inline-block', 'position': 'relative', 'float':'right'}

Stations = pd.read_excel('SNOTELStationNames&Triplets.xlsx')
station_names = Stations.loc[:,'Station Name'].tolist()
triplets = Stations.loc[:, 'Station Triplet'].tolist()

options = []
for i in range(len(Stations.index)):
  options.append({'label': station_names[i], 'value': triplets[i]})
           
parameter_options =  [{'label': 'WTEQ', 'value': 'WTEQ'},
                      {'label': 'PREC', 'value': 'PREC'},
                      {'label': 'PRCP', 'value': 'PRCP'},
                      {'label': 'SNWD', 'value': 'SNWD'},
                      {'label': 'TAVG', 'value': 'TAVG'},
                      {'label': 'TOBS', 'value': 'TOBS'},
                      {'label': 'TMAX', 'value': 'TMAX'},
                      {'label': 'TMIN', 'value': 'TMIN'}]

#----------------------------------------------------------------------

# Initialize the app class

app = dash.Dash(__name__)
#https://stackoverflow.com/questions/50844844/python-dash-custom-css
#----------------------------------------------------------------------

# app layout

app.layout = html.Div(children = [
    
    html.Div(children = [
        html.Br(),
        html.Label(['Select the station to be used as the response:'], style=subheading),
        dcc.Dropdown(id = 'response-station', 
                     options = options, 
                     multi=False,
                     value = 'response',
#                      style = station_dropdown_style
                    ),
        dcc.Dropdown(id = 'response-parameter', 
                    options = parameter_options,
                    multi=False,
                    value="parameter",
#                     style = parameter_dropdown_style
                    )]),    
    
    html.Div(children = [
        html.Label(['Select station parameter pairs to be used as predictors:'], style=subheading),
        html.Br(),
        dcc.Dropdown(id = 'predictor-station1', 
                     options = options, 
                     multi = False,
                     value = 'predictor',
#                      style = station_dropdown_style,
                    ),
        dcc.Dropdown(id = 'predictor-parameter1', 
                    options = parameter_options,
                    multi = False,
                    value="parameter",
#                     style = parameter_dropdown_style,
                    ),
                       
        html.Br(),
        dcc.Dropdown(id = 'predictor-station2', 
                     options = options, 
                     multi = False,
                     value = 'predictor',
#                      style = station_dropdown_style
                    ),
        dcc.Dropdown(id = 'predictor-parameter2', 
                    options = parameter_options,
                    multi = False,
                    value = "parameter",
#                     style = parameter_dropdown_style,
                    ),
    
                     
        html.Br(),
        dcc.Dropdown(id = 'predictor-station3', 
                     options = options, 
                     multi = False,
                     value = 'predictor',
#                      style = station_dropdown_style
                    ),
        dcc.Dropdown(id = 'predictor-parameter3', 
                    options = parameter_options,
                    multi = False,
                    value="parameter",
#                     style = 'station_dropdown_style'
                    ),
    ]),
    html.Div(children = [
    
        html.Br(),
        html.Label(['Select data range to train the regression model: '], style=subheading),
        html.Br(),
        dcc.DatePickerRange(
            id='date-picker-range',
            start_date_placeholder_text='Select a start date!',
            end_date_placeholder_text='Select an end date!',
#             style={'width': '50%', 'display': 'inline-block', 'vertical-align': 'left','border-bottom': 'solid 3px', 
#                   'border-color':'black','padding-top': '50px'}
        ),
    ]),
    

    html.Div(children = [
            dcc.Graph(id='training vs test plot', style={'width': '80%','padding-left':'2px','float':'right'}),
            dcc.Graph(id='regression model plot',style={'width': '80%','padding-left':'2px','float':'right'}),
        
        html.Br(),
        html.Button(id='submit-button-state',
                   children = 'Submit',
#                    style={'fontsize:24'}
        )
        
        ])

        ])

#----------------------------------------------------------------------

# callback for regression plots

# @app.callback(Output('training vs test plot','figure'),
#               [Input('response-station-picker','response'),
#                Input('predictor-station-picker','predictors'),
#                Input('parameter-picker','parameter'),
#                Input('date-picker-range', 'start_date'),
#                Input('date-picker-range', 'end_date')],
#              )

                
@app.callback(Output('training vs test plot','figure'),
              Input('submit-button-state', 'children'),         
               State('response-station','response'),
               State('response-parameter','response'),
               State('predictor-station1','predictor'),
               State('predictor-station2','predictor'),
               State('predictor-station3','predictor'),               
               State('predictor-parameter1','response'),
               State('predictor-parameter2','response'),
               State('predictor-parameter3','response'),
               State('date-picker-range', 'start_date'),
               State('date-picker-range', 'end_date'),
 )
             
             
                
                
# @app.callback(Output('regression model plot',component_property='figure'),
#               [Input('response-station-picker',component_property='response'),
#                Input('predictor-station-picker',component_property='predictors'),
#                Input('parameter-picker',component_property='parameter'),
#                Input('date-picker-range', 'start_date'),
#                Input('date-picker-range', 'end_date')
#               ])
             
def plots(response, predictors, parameter, start_date, end_date):
    return 5
        
        
        
    






#----------------------------------------------------------------------------------------------------------
#Run the app

if __name__ == '__main__':
              app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Oct/2021 20:39:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2021 20:39:38] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2021 20:39:38] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2021 20:39:38] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2021 20:39:38] "GET /_dash-component-suites/dash/dcc/async-datepicker.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2021 20:39:38] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2021 20:39:38] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
